### Essential Libraries

In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
from torch.utils.data import DataLoader
from dataloader import ATeX  # Ensure dataloader.py is in the same directory or in PYTHONPATH
import torchvision.models as models
from sklearn.metrics import classification_report  # Import the function
from torch.optim import SGD
import numpy as np
from torchvision.models import efficientnet_b0
from sklearn.metrics import accuracy_score
from torch.optim.lr_scheduler import CosineAnnealingLR

### Dataset Preparation and DataLoader Setup

In [2]:
# Define transformations and dataset
mean_std = ([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
data_transforms = transforms.Compose([
    transforms.ToTensor(),
    transforms.Normalize(mean_std[0], mean_std[1]),
])

train_dataset = ATeX(split="train", transform=data_transforms)
test_dataset = ATeX(split="test", transform=data_transforms)

# Define DataLoaders
batch_size = 64
train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

# Validation Dataset and DataLoader
val_dataset = ATeX(split="val", transform=data_transforms)
val_loader = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

### Define the CNN model

In [3]:
# Load pre-trained EfficientNet-B0 model and modify for 15 classes
class EffNetB0(nn.Module):
    def __init__(self, num_classes=15):
        super(EffNetB0, self).__init__()
        self.effnet = efficientnet_b0(pretrained=True)
        # Replace the classifier with a new one for 15 classes
        self.effnet.classifier[1] = nn.Linear(self.effnet.classifier[1].in_features, num_classes)

    def forward(self, x):
        return self.effnet(x)

# Instantiate the model
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = EffNetB0(num_classes=15).to(device)


c:\Users\Roja\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
c:\Users\Roja\AppData\Local\Programs\Python\Python312\Lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


### Training, Validation and Evaluation

In [4]:
optimizer = torch.optim.SGD(model.parameters(), lr=0.01, momentum=0.9, weight_decay=1e-4)
scheduler = CosineAnnealingLR(optimizer, T_max=30)  # Cosine annealing over 30 epochs
# Define the loss function (CrossEntropyLoss or Label Smoothing)
criterion = nn.CrossEntropyLoss()  # Replace with label_smoothing_loss if needed

num_epochs = 30
for epoch in range(num_epochs):
    # Training Phase
    model.train()
    running_loss = 0.0
    for images, labels, _ in train_loader:
        images, labels = images.to(device), labels.to(device)

        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)  # Use the defined loss function

        # Backward pass and optimization
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

    # Calculate average training loss
    train_loss = running_loss / len(train_loader)

    # Update learning rate
    scheduler.step()

    # Validation Phase
    model.eval()
    val_loss = 0.0
    all_val_labels = []
    all_val_preds = []
    with torch.no_grad():
        for images, labels, _ in val_loader:
            images, labels = images.to(device), labels.to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            val_loss += loss.item()

            _, preds = torch.max(outputs, 1)
            all_val_labels.extend(labels.cpu().numpy())
            all_val_preds.extend(preds.cpu().numpy())

    # Calculate validation metrics
    val_loss /= len(val_loader)
    val_report = classification_report(
        all_val_labels, all_val_preds, target_names=train_dataset.classes, zero_division=0, output_dict=True
    )
    val_precision = val_report['weighted avg']['precision']
    val_recall = val_report['weighted avg']['recall']
    val_f1 = val_report['weighted avg']['f1-score']

    # Print training and validation metrics
    print(f"Epoch [{epoch+1}/{num_epochs}]: Train Loss: {train_loss:.4f}, "
          f"Val Loss: {val_loss:.4f}, Precision: {val_precision:.4f}, "
          f"Recall: {val_recall:.4f}, F1-Score: {val_f1:.4f}")


Epoch [1/30]: Train Loss: 2.0953, Val Loss: 1.6444, Precision: 0.4908, Recall: 0.4952, F1-Score: 0.4703
Epoch [2/30]: Train Loss: 1.4557, Val Loss: 1.1627, Precision: 0.6265, Recall: 0.6118, F1-Score: 0.6071
Epoch [3/30]: Train Loss: 1.0522, Val Loss: 0.8320, Precision: 0.7487, Recall: 0.7412, F1-Score: 0.7379
Epoch [4/30]: Train Loss: 0.8547, Val Loss: 0.7124, Precision: 0.7808, Recall: 0.7740, F1-Score: 0.7748
Epoch [5/30]: Train Loss: 0.6690, Val Loss: 0.7057, Precision: 0.7669, Recall: 0.7604, F1-Score: 0.7598
Epoch [6/30]: Train Loss: 0.5663, Val Loss: 0.5448, Precision: 0.8459, Recall: 0.8450, F1-Score: 0.8446
Epoch [7/30]: Train Loss: 0.4864, Val Loss: 0.7107, Precision: 0.8255, Recall: 0.8187, F1-Score: 0.8184
Epoch [8/30]: Train Loss: 0.4359, Val Loss: 0.4873, Precision: 0.8526, Recall: 0.8482, F1-Score: 0.8482
Epoch [9/30]: Train Loss: 0.3534, Val Loss: 0.4541, Precision: 0.8556, Recall: 0.8538, F1-Score: 0.8531
Epoch [10/30]: Train Loss: 0.2980, Val Loss: 0.4171, Precision: 

### Testing and Evaluation

In [5]:
# Testing the model
model.eval()
all_labels = []
all_preds = []

with torch.no_grad():
    for images, labels, _ in test_loader:  # Assuming your DataLoader provides 3 items
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        _, preds = torch.max(outputs, 1)
        
        all_labels.extend(labels.cpu().numpy())
        all_preds.extend(preds.cpu().numpy())

# Calculate classification metrics
class_names = train_dataset.classes  # Assuming train_dataset has the class names
report = classification_report(all_labels, all_preds, target_names=class_names)
print("Classification Report:\n")
print(report)

# Compute per-class accuracy
all_labels = np.array(all_labels)
all_preds = np.array(all_preds)



Classification Report:

              precision    recall  f1-score   support

       delta       0.96      0.97      0.96       330
     estuary       0.93      0.96      0.94       125
       flood       0.93      0.90      0.91       283
    glaciers       0.97      0.97      0.97       240
  hot_spring       0.88      0.86      0.87       201
        lake       0.87      0.86      0.87       153
        pool       0.97      0.92      0.95        79
      puddle       0.87      0.77      0.82       168
      rapids       0.88      0.90      0.89       219
       river       0.81      0.88      0.84        73
         sea       0.90      0.96      0.93       108
        snow       0.83      0.87      0.85       142
       swamp       0.96      0.95      0.95       188
   waterfall       0.69      0.75      0.72        96
     wetland       0.99      0.97      0.98        93

    accuracy                           0.91      2498
   macro avg       0.90      0.90      0.90      2498
we

### Save Results

In [6]:
# Save classification report
with open("classification_report_EfficientNet-B0 model.txt", "w") as f:
    f.write(report)

# Save model weights
torch.save(model.state_dict(), "EfficientNet-B0 model_cpu.pth")